In [1]:
import pandas as pd
# Date
import datetime as dt
# Settings
import warnings

In [6]:
df = pd.read_csv('/content/online_retail_II.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


In [8]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [9]:
df = df[~df["Invoice"].str.contains("C", na = False)]
# Removing missing values from the dataset
df.dropna(inplace = True)

In [10]:
df["InvoiceDate"].max()

'2011-12-09 12:50:00'

In [11]:
today_date = dt.datetime(2011,12,9)

In [12]:
df["Customer ID"] = df["Customer ID"].astype(int)

In [13]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

In [14]:
recency = (today_date - df.groupby("Customer ID").agg({"InvoiceDate":"max"}))
# Rename column name as Recency
recency.rename(columns = {"InvoiceDate":"Recency"}, inplace = True)
# Change the values to day format
recency_df = recency["Recency"].apply(lambda x: x.days)
recency_df.head()

Customer ID
12346    324
12347      1
12348     74
12349     17
12350    309
Name: Recency, dtype: int64

In [15]:
freq_df = df.groupby("Customer ID").agg({"InvoiceDate":"nunique"})
# Rename column name as Frequency
freq_df.rename(columns={"InvoiceDate": "Frequency"}, inplace=True)
freq_df.head()

,Frequency
Customer ID,
12346,12
12347,8
12348,5
12349,4
12350,1


In [16]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [17]:
# Grouping and sum up total prices according to each Customer ID
monetary_df = df.groupby("Customer ID").agg({"TotalPrice":"sum"})
# Rename Total Price column as Monetary
monetary_df.rename(columns={"TotalPrice":"Monetary"}, inplace=True)
monetary_df.head()

,Monetary
Customer ID,
12346,77556.46
12347,5633.32
12348,2019.40
12349,4428.69
12350,334.40


In [18]:
rfm = pd.concat([recency_df, freq_df, monetary_df],  axis=1)
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346,324,12,77556.46
12347,1,8,5633.32
12348,74,5,2019.40
12349,17,4,4428.69
12350,309,1,334.40


In [19]:
# Dividing the recency values into recency scores such that the lowest recency value as 5 and the highest as 1
rfm["RecencyScore"] = pd.qcut(rfm["Recency"], 5, labels = [5, 4 , 3, 2, 1])
# Dividing the frequency values into frequency scores such that the lowest frequency value as 1 and the highest as 5
rfm["FrequencyScore"]= pd.qcut(rfm["Frequency"].rank(method="first"),5, labels=[1,2,3,4,5])
# Dividing the monetary values into monetary scores such that the lowest monetary value as 1 and the highest as 5
rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels = [1, 2, 3, 4, 5])

In [20]:
# Combining Recency, Frequency, and Monetary Scores in a string format
rfm["RFM_SCORE"] = (rfm['RecencyScore'].astype(str) +
                    rfm['FrequencyScore'].astype(str) +
                    rfm['MonetaryScore'].astype(str))

In [21]:
# Customers with best scores
rfm[rfm["RFM_SCORE"]=="555"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12362,2,11,5356.23,5,5,5,555
12395,18,15,5067.27,5,5,5,555
12417,2,20,6816.91,5,5,5,555
12433,-1,10,20581.26,5,5,5,555
12437,0,39,12683.40,5,5,5,555


In [22]:
# Customers with worst scores
rfm[rfm["RFM_SCORE"]=="111"].head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12387,414,1,143.94,1,1,1,111
12392,590,1,234.75,1,1,1,111
12400,413,1,205.25,1,1,1,111
12404,681,1,63.24,1,1,1,111
12416,655,1,202.56,1,1,1,111


In [23]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

In [24]:
# Recency and Frequency scores are turned into string format, combined and assigned to Segment
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
# Segments are changed with the definitons of seg_map
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)

In [25]:
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346,324,12,77556.46,2,5,5,255,Can't Loose
12347,1,8,5633.32,5,4,5,545,Champions
12348,74,5,2019.40,3,4,4,344,Loyal Customers
12349,17,4,4428.69,5,3,5,535,Potential Loyalists
12350,309,1,334.40,2,1,2,212,Hibernating


In [26]:
rfm[["Segment","Recency","Frequency", "Monetary"]].groupby("Segment").agg(["mean","median","count"])

Recency               Frequency               \
                           mean median count       mean median count   
Segment                                                                
About to Sleep       105.589147   92.0   387   1.359173    1.0   387   
At Risk              371.632000  375.0   750   3.898667    4.0   750   
Can't Loose          332.861111  324.5    72  15.694444   11.0    72   
Champions              7.255294    7.0   850  19.154118   11.5   850   
Hibernating          458.375575  434.0  1523   1.252791    1.0  1523   
Loyal Customers       66.274390   52.0  1148   9.776132    8.0  1148   
Need Attention       112.259259  105.0   270   3.148148    3.0   270   
New Customers          9.368421    9.0    57   1.000000    1.0    57   
Potential Loyalists   24.235294   22.0   714   2.588235    3.0   714   
Promising             37.281818   36.5   110   1.000000    1.0   110   

                         Monetary                  
                             mean    median count  
Segment                                            
About to Sleep         534.749749   371.010   387  
At Risk               1379.649893   961.295   750  
Can't Loose           8278.154333  3870.535    72  
Champions            10816.020971  4017.555   850  
Hibernating            437.797546   285.110  1523  
Loyal Customers       4196.920353  2601.895  1148  
Need Attention        1276.347556   974.970   270  
New Customers          350.007719   255.400    57  
Potential Loyalists   1158.276598   692.520   714  
Promising              324.497000   221.620   110